In [12]:
from binance.spot import Spot as sp
import inspect as i
import json
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pytz
from plotly.subplots import make_subplots
from finta import TA as ta

tz_local = pytz.timezone("Europe/Zurich")

apiKey = "OY6oBQD3qm6VGzokI29UaCm8hrnpCrjv4KEpCY45UasLAj7D3xkLPGwFjNNz1neL"
apiKey_secret = "D82a2gKS5lGbmFRvkPP9PEQTwnfXKfxFBNddOVrjixQ8MJkSirLxuRyNoenO3JVj"

baseURL = "https://testnet.binance.vision"

pd.options.display.float_format = "{:,.3f}".format

In [13]:
#API SETUP
sp_client = sp(key=apiKey, secret=apiKey_secret)

#get Different cryptos
dict_symbols = sp_client.exchange_info()

#API CALL
api_interval = "1m"
pd_interval = "T"
api_limit = 1000
end_time = dt.datetime.now()
end_time_api = int(round(end_time.timestamp()*1000,0))

dict_data = sp_client.klines(endTime=end_time_api, symbol="ADATUSD", interval="1m", limit=api_limit)

In [14]:
df_symbols = pd.json_normalize(dict_symbols["symbols"])

In [15]:
headers_df = ["openTime","open", "high","low","close","vol","closeTime","quiteAssetVolume","trades_qty","baseAssetVol_taker","baseQuoteVol_taker","ignore"]

df_data_raw = pd.DataFrame(dict_data, columns=headers_df)
#convert to datetimes
df_data_raw["openTime"] = pd.to_datetime(df_data_raw["openTime"]/1000, unit="s")
df_data_raw["closeTime"] = pd.to_datetime(df_data_raw["closeTime"]/1000, unit="s")

#round to nearest second
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.floor("S")

#change timezone

#reset index to opentime
df_data_raw.reset_index(inplace=True,drop=True)
df_data_raw.set_index("openTime", inplace=True)

# convert all none-date columns to float
cols = df_data_raw.select_dtypes(exclude=["datetime64"]).columns
df_data_raw[cols] = df_data_raw[cols].apply(pd.to_numeric, downcast="float",errors="coerce")

# mark TZ as UTC then change to CET
df_data_raw.index = df_data_raw.index.tz_localize("UTC").tz_convert("CET")
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.tz_localize("UTC").dt.tz_convert("CET")


In [16]:
df_d = pd.read_csv("./app/binance_ws_comp2.csv")
df_d["timestamp"] = pd.to_datetime(df_d["timestamp"])
df_d.reset_index(drop=True, inplace=True)
df_d.set_index("timestamp", inplace=True)

df_d.index = df_d.index.tz_localize("UTC").tz_convert("CET")
df_d.head()

,open,high,low,close,bid,ask
timestamp,,,,,,
2021-08-22 14:20:02+02:00,"48,660.010","49,757.040","48,562.800","49,021.300","49,021.300","49,021.310"
2021-08-22 14:20:03+02:00,"48,660.010","49,757.040","48,562.800","49,021.300","49,021.300","49,021.310"
2021-08-22 14:20:04+02:00,"48,660.010","49,757.040","48,562.800","49,021.310","49,021.300","49,021.310"
2021-08-22 14:20:05+02:00,"48,653.580","49,757.040","48,562.800","49,021.310","49,021.300","49,021.310"
2021-08-22 14:20:06+02:00,"48,653.580","49,757.040","48,562.800","49,021.300","49,021.300","49,021.310"


In [17]:
r_int = 10

df_d_r = df_d.resample("{}S".format(r_int), closed="right").max()
df_d_r.head()

,open,high,low,close,bid,ask
timestamp,,,,,,
2021-08-22 14:20:00+02:00,"48,660.010","49,757.040","48,562.800","49,021.310","49,021.300","49,021.310"
2021-08-22 14:20:10+02:00,"48,650.010","49,757.040","48,562.800","49,014.580","49,014.570","49,014.580"
2021-08-22 14:20:20+02:00,"48,653.580","49,757.040","48,562.800","49,014.550","49,014.550","49,014.560"
2021-08-22 14:20:30+02:00,"48,648.300","49,757.040","48,562.800","49,013.560","49,013.550","49,013.560"
2021-08-22 14:20:40+02:00,"48,642.860","49,757.040","48,562.800","49,013.560","49,013.550","49,013.560"


In [18]:
#add rolling averages

wd_short = 5
wd_medium = 20
wd_medium_ema = 13
wd_long = 200

df_close = df_d_r[["close"]].copy()
df_close["mav_short"] = df_close["close"].rolling(window=wd_short).mean()
df_close["mav_medium"] = df_close["close"].rolling(window=wd_medium).mean()
df_close["mav_long"] = df_close["close"].rolling(window=wd_long).mean()

df_close["ema_short"] = df_close["close"].ewm(span=wd_short,  ignore_na=True).mean()
df_close["ema_med"] = df_close["close"].ewm(span=wd_medium_ema,  ignore_na=True).mean()

In [19]:
# Bollinger Bands
df_close["bohlinger_upper"] = df_close["mav_medium"] + (2*(df_close["mav_medium"].rolling(window=wd_medium).std()))
df_close["bohlinger_lower"] = df_close["mav_medium"] - (2*(df_close["mav_medium"].rolling(window=wd_medium).std()))


In [20]:
# Stochastic Oscillator

so_wd_k = 14
so_wd_d = 3

df_close["k_line"] = ((df_close["close"] - df_close["close"].rolling(window=so_wd_k).min()) * 100 / 
    (df_close["close"].rolling(window=so_wd_k).max() - df_close["close"].rolling(window=so_wd_k).min()))

df_close["d_line"] = df_close["k_line"].rolling(window=so_wd_k).mean()

df_close["so_20"] = 20
df_close["so_80"] = 80

In [21]:
# MACD
macd_fast = 26
macd_slow = 12
macd_smooth = 9

df_close["macd_ema_fast"] = df_close["close"].ewm(span=macd_fast, adjust=False, ignore_na=True).mean()
df_close["macd_ema_slow"] = df_close["close"].ewm(span=macd_slow, adjust=False, ignore_na=True).mean()
df_close["macd_base"] = df_close["macd_ema_slow"] - df_close["macd_ema_fast"]
df_close["macd_signal"] = df_close["macd_base"].ewm(span=macd_smooth, adjust=False, ignore_na=True).mean()
df_close["macd_hist"] = df_close["macd_base"] - df_close["macd_signal"]
df_close["macd_color"] = np.where(df_close["macd_hist"]<0, "red", "green")

In [22]:
def comp_df(df, col1, col2, metric):
    comp = str(metric+"_comp")
    
    df[comp] = np.where(df[col1] > df[col2], 1,-1)
    return df


comp_df(df_close,"ema_short","ema_med","ema")
    


ema_conditions = [
    ((df_close["ema_short"].shift(3) > df_close["ema_med"].shift(3)) & (df_close["ema_short"].shift(4) < df_close["ema_med"].shift(4)) & (abs(df_close["ema_comp"].rolling(window=3).sum()) == 3)),
    ((df_close["ema_short"].shift(3) < df_close["ema_med"].shift(3)) & (df_close["ema_short"].shift(4) > df_close["ema_med"].shift(4)) & (abs(df_close["ema_comp"].rolling(window=3).sum()) == 3))  
]

ema_choices = [1,-1]

df_close["ema_switch"] = np.select(ema_conditions, ema_choices, default=0)

df_marks = df_close.loc[df_close["ema_switch"].isin([1,-1])]

In [23]:
def udf_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0]
    return slope


df_close["slope_close"] = df_close["close"].rolling(4, min_periods=4).apply(udf_slope)
df_close["slope_close_ewm"] = df_close["slope_close"].ewm(span=3, ignore_na=True).mean()

In [27]:
i_rows = 4

fig2 = make_subplots(rows=i_rows, cols=1, shared_xaxes=True, vertical_spacing=0.01
                    , specs=[[{}],[{}],[{}],[{"secondary_y": True}]])

### ---------------> OHLC
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)
), row=1, col=1)          



### ---------------> ema_short
fig2.add_trace(go.Scatter(
    mode ="lines"
    , x=df_close.index
    , y=df_close["ema_short"]
    , line = dict(color="blue")
    , name="ema_{}".format(wd_short)
), row=1, col=1)

### ---------------> ema_med
fig2.add_trace(go.Scatter(
    mode ="lines"
    , x=df_close.index
    , y=df_close["ema_med"]
    , line = dict(color="yellow")
    , name="ema_{}".format(wd_medium_ema)
), row=1, col=1)


######### 3rd graph
### ---------------> BASE CLOSE

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)

    ), row=2, col=1
)

### ---------------> Bohlinger Up
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["bohlinger_upper"]
        , line=dict(dash="dash", color="green")
        , name="bollinger_up"
    ), row=2, col=1
)

### ---------------> Bohlinger Low
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["bohlinger_lower"]
        , line=dict(dash="dash", color="green")
        , name="bollinger_low"
    ), row=2, col=1
)

######### 4th graph
### ---------------> MACD

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["macd_base"]
        , name="MACD"
        , line=dict(color="black", width=2)


    ), row=3, col=1
)

### ---------------> MACD SIGNAL

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["macd_signal"]
        , name="macd_signal"
        , line=dict(color="lightblue", width=2, dash="dot")

    ), row=3, col=1
)

### ---------------> MACD HIST

fig2.add_trace(
    go.Bar(
        x=df_close.index
        , y=df_close["macd_hist"]
        , name="macd_hist"
        , marker_color=df_close["macd_color"]

    ), row=3, col=1
)

######### 6th graph
### ---------------> BASE CLOSE

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["close"]
        , name="CLOSE"
        , line=dict(color="black", width=2)
        , yaxis="y4"

    ), row=4, col=1
)

### ---------------> %K
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["k_line"]
        , name="%K"
        , line=dict(color="red", width=2, dash="solid")
        

    ), row=4, col=1, secondary_y=True
)

### ---------------> %D
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["d_line"]
        , name="%D"
        , line=dict(color="orange", width=2, dash="solid")
        

    ), row=4, col=1, secondary_y=True
)

### ---------------> SO bounds
fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["so_20"]
        , name="bound_20"
        , line=dict(color="blue", width=0.5, dash="dash")
    ), row=4, col=1, secondary_y=True
)

fig2.add_trace(
    go.Scatter(
        mode="lines"
        , x=df_close.index
        , y=df_close["so_80"]
        , name="bound_80"
        , line=dict(color="blue", width=0.5, dash="dash")
    ), row=4, col=1, secondary_y=True
)




for i in range(1,i_rows+1):
    
    if i != 3:
        fig2.add_trace(
        go.Scatter(
            mode="lines"
            , x=df_d.index
            , y=df_d["close"]
            , name="CLOSE"
            , line=dict(color="purple", width=1)
        ), row=i, col=1)    





rn_yaxis_a = (df_close["close"].min()) * 1.001
rn_yaxis_b = (df_close["close"].max()) * 1.001

fig2.update_layout(
    width=1500
    , height=2000
    , xaxis1 = dict(rangeslider=dict(visible=False))
    , xaxis6 = dict(rangeslider=dict(visible=True))

    , yaxis1 = dict(range=[rn_yaxis_a,rn_yaxis_b])
    , yaxis2 = dict(range=[rn_yaxis_a,rn_yaxis_b])
    , yaxis4 = dict(range=[rn_yaxis_a,rn_yaxis_b])



)


for idx, x in df_marks.iterrows():
    x_date = idx.strftime("%Y-%m-%d %H:%M:%S")
    fig2.add_shape(go.layout.Shape(type="line", xref="x", x0=x_date, x1=x_date, y0=-10000, y1=100000, line={"color":("green" if x["ema_switch"] == 1 else "red"), "width":2, "dash":"dot"}), row=[1,2,4], col="all")



fig2.show()

In [ ]:
df_close[-100:].to_clipboard(excel=True)

In [ ]:
df_ohlc = df_data_dtFilter[["open","high","low","close"]].copy()

df_ohlc["F_ema_5"] = ta.EMA(df_ohlc, 5)
df_ohlc["F_ema_13"] = ta.EMA(df_ohlc, 13)
df_ohlc[-20:]

,open,high,low,close,F_ema_5,F_ema_13
dateTime,,,,,,
2021-08-22 14:36:00+02:00,2.597,2.597,2.585,2.588,2.585,2.589
2021-08-22 14:37:00+02:00,2.586,2.586,2.586,2.586,2.586,2.589
2021-08-22 14:38:00+02:00,2.586,2.586,2.586,2.586,2.586,2.589
2021-08-22 14:39:00+02:00,2.586,2.586,2.586,2.586,2.586,2.588
2021-08-22 14:40:00+02:00,2.585,2.585,2.585,2.585,2.586,2.588
2021-08-22 14:41:00+02:00,2.585,2.585,2.582,2.582,2.584,2.587
2021-08-22 14:42:00+02:00,2.582,2.582,2.578,2.578,2.582,2.586
2021-08-22 14:43:00+02:00,2.578,2.578,2.578,2.578,2.581,2.585
2021-08-22 14:44:00+02:00,2.578,2.578,2.573,2.573,2.578,2.583
